TODOOs
- clean up code
- learning rate discussion
- make a consisten train and test df

## A notebook for parameter tuning

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import numpy as np
import pandas as pd
import torch.nn.functional as F
import gensim
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 1000)

In [2]:
# Define the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    print("Using GPU:")
    print(torch.cuda.get_device_name(0))  # 0 is the index of the GPU
else:
    print("Using CPU")

Using GPU:
Quadro T1000 with Max-Q Design


Read Inputs

In [3]:
df_news = pd.read_pickle('..\input\df_news.pkl')

## Simple feed-forward neural network for binary classification.

Parameters to tune:
 - learning rate
 - more epochs

### 1. Testing improved accuracy with the extra "subject" category

Firstly using the category

In [20]:
from torch.utils.data import Dataset, DataLoader # this is for data loading
from sklearn.model_selection import train_test_split

class VectorizedTextDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        feature = torch.tensor(self.features[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return feature, label
    
df_training = df_news[['vector', 'subject', 'Real']]
label_encoder = LabelEncoder()
df_training['subject'] = label_encoder.fit_transform(df_training['subject'])

# Convert features to numpy arrays
# X = np.array(df_training['vector'].tolist()) # features - array of the vectors 
X_vectors = np.array(df_training['vector'].tolist())
df_features = df_training.drop(columns=['vector', 'Real'])

# Convert features to numpy array
X_features = df_features.values

# Combine word2vec vectors with features
X = np.hstack([X_vectors, X_features])
y = np.array(df_training['Real'].tolist()) # label - array of the 1 or 0s for each vector

# Split data into test and train - here we do an 80/20 split. Optional: additional validation set 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create Dataset instances
train_dataset = VectorizedTextDataset(X_train, y_train) # get the data in a structured dataset (abstract class that wraps around). 
test_dataset = VectorizedTextDataset(X_test, y_test)

# Create DataLoader instances
# this is for loading the data onto the GPU
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

class TextClassifier(nn.Module): # create a class with the neural networ module
    def __init__(self, input_dim): # input dimensions
        super(TextClassifier, self).__init__()
        self.fc = nn.Linear(input_dim, 1) # nn.Linear (input, output) - dimensions of input (features) and output (labels)
        self.sigmoid = nn.Sigmoid() # maps the output onto a 0 to 1 line, representing the probability of it belonging to a 1 
    
    def forward(self, x):  # forward nn 
        x = self.fc(x) # first the input is passed through the fully connected linear layer 
        x = self.sigmoid(x) # then it goes through the sigmoid 
        return x
    
# Model, Loss, Optimizer
input_dim = X_train.shape[1]  # Number of features - 100 since this is the # components of the vector
model = TextClassifier(input_dim)
criterion = nn.BCELoss()  # loss function minimise 
optimizer = optim.Adam(model.parameters(), lr=0.001)  # optimizer - lr learning rate, a critical hyperparameter that controls how large or small the updates to the weights will be at each step
# Adam optimizer

# Training loop
for epoch in range(5):  # choose epochs - one complete pass through the entire dataset
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad() #  reset the gradients for each new epoch 
        outputs = model(features) 
        loss = criterion(outputs.squeeze(), labels.float())  # compute the loss
        loss.backward()
        optimizer.step() # update the weights 
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

def evaluate_model(model, loader):
    model.eval() # model is in evaluate mode
    correct = 0
    total = 0
    with torch.no_grad():
        for features, labels in loader: # the loader is the dataset
            outputs = model(features) # get the outputs
            predicted = (outputs.squeeze() > 0.5).long()  # predicted label - it is assigned to whichever has a probability >0.5 (threshold can be changed)
            correct += (predicted == labels).sum().item() 
            total += labels.size(0)
    accuracy = correct / total
    return accuracy

print(f'Train Accuracy: {evaluate_model(model, train_loader)}')
print(f'Test Accuracy: {evaluate_model(model, test_loader)}')

C:\Users\jlari\AppData\Local\Temp\ipykernel_32320\3103970089.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_training['subject'] = label_encoder.fit_transform(df_training['subject'])


Epoch 1, Loss: 0.06220392882823944
Epoch 2, Loss: 0.4582524299621582
Epoch 3, Loss: 0.001292006578296423
Epoch 4, Loss: 0.0012848366750404239
Epoch 5, Loss: 0.002674767514690757
Train Accuracy: 0.9583774152235648
Test Accuracy: 0.9587973273942093


Now not using the category (this is the final model saved as forward_nn)

FOR A FAIR COMPARISON YOU WOULD NEED TO USE THE SAME TRAIN AND TEST SET EACH TIME, OR RETRAIN THE MODEL

In [23]:
# reload model 

# Define the model class
class TextClassifier_nosbj(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(TextClassifier_nosbj, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)
    
    def forward(self, x):
        x = self.fc(x)
        return x

# Initialize model
input_dim = 100  # Set this to your input dimension
num_classes = 2  # Binary classification
model_nosbj = TextClassifier_nosbj(input_dim, num_classes)
# encode the "subject" feature
# Convert features to numpy arrays
X_nosb = np.array(df_training['vector'].tolist())  # Features - array of vectors
y_nosb = np.array(df_training['Real'].tolist())    # Labels - array of 1s or 0s

# Split data into train and test sets
X_train_nosb, X_test_nosb, y_train_nosb, y_test_nosb = train_test_split(X_nosb, y_nosb, test_size=0.2, random_state=42)

# Create Dataset instances
test_dataset_nosb = VectorizedTextDataset(X_test_nosb, y_test_nosb)

# Create DataLoader instances
test_loader_nosb = DataLoader(test_dataset_nosb, batch_size=2, shuffle=False)
# Load the model
model_nosbj.load_state_dict(torch.load(r'..\models\forward_nn.pth'))
model_nosbj.eval()

model_nosbj.eval() # model is in evaluate mode
correct = 0
total = 0
with torch.no_grad():
    for features, labels in test_loader_nosb: # the loader is the dataset
        outputs = model_nosbj(features) # get the outputs
        predicted = (outputs.squeeze() > 0.5).long()  # predicted label - it is assigned to whichever has a probability >0.5 (threshold can be changed)
        correct += (predicted == labels).sum().item() 
        total += labels.size(0)
accuracy = correct / total

C:\Users\jlari\AppData\Local\Temp\ipykernel_32320\681567642.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_nosbj.load_state_dict(torch.load(r'..\models\forward_n

Changing the learning rate:

In [27]:
from torch.utils.data import Dataset, DataLoader # this is for data loading
from sklearn.model_selection import train_test_split

class VectorizedTextDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        feature = torch.tensor(self.features[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return feature, label
    
df_training = df_news[['vector', 'subject', 'Real']]
label_encoder = LabelEncoder()
df_training['subject'] = label_encoder.fit_transform(df_training['subject'])

# Convert features to numpy arrays
# X = np.array(df_training['vector'].tolist()) # features - array of the vectors 
X_vectors = np.array(df_training['vector'].tolist())
df_features = df_training.drop(columns=['vector', 'Real'])

# Convert features to numpy array
X_features = df_features.values

# Combine word2vec vectors with features
X = np.hstack([X_vectors, X_features])
y = np.array(df_training['Real'].tolist()) # label - array of the 1 or 0s for each vector

# Split data into test and train - here we do an 80/20 split. Optional: additional validation set 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create Dataset instances
train_dataset = VectorizedTextDataset(X_train, y_train) # get the data in a structured dataset (abstract class that wraps around). 
test_dataset = VectorizedTextDataset(X_test, y_test)

# Create DataLoader instances
# this is for loading the data onto the GPU
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=False)

class TextClassifier(nn.Module): # create a class with the neural networ module
    def __init__(self, input_dim): # input dimensions
        super(TextClassifier, self).__init__()
        self.fc = nn.Linear(input_dim, 1) # nn.Linear (input, output) - dimensions of input (features) and output (labels)
        self.sigmoid = nn.Sigmoid() # maps the output onto a 0 to 1 line, representing the probability of it belonging to a 1 
    
    def forward(self, x):  # forward nn 
        x = self.fc(x) # first the input is passed through the fully connected linear layer 
        x = self.sigmoid(x) # then it goes through the sigmoid 
        return x
    
# Model, Loss, Optimizer
input_dim = X_train.shape[1]  # Number of features - 100 since this is the # components of the vector
model = TextClassifier(input_dim)
criterion = nn.BCELoss()  # loss function minimise 
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # optimizer - lr learning rate, a critical hyperparameter that controls how large or small the updates to the weights will be at each step
# Adam optimizer

# Training loop
for epoch in range(10):  # choose epochs - one complete pass through the entire dataset
    model.train()
    for features, labels in train_loader:
        optimizer.zero_grad() #  reset the gradients for each new epoch 
        outputs = model(features) 
        loss = criterion(outputs.squeeze(), labels.float())  # compute the loss
        loss.backward()
        optimizer.step() # update the weights 
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

def evaluate_model(model, loader):
    model.eval() # model is in evaluate mode
    correct = 0
    total = 0
    with torch.no_grad():
        for features, labels in loader: # the loader is the dataset
            outputs = model(features) # get the outputs
            predicted = (outputs.squeeze() > 0.5).long()  # predicted label - it is assigned to whichever has a probability >0.5 (threshold can be changed)
            correct += (predicted == labels).sum().item() 
            total += labels.size(0)
    accuracy = correct / total
    return accuracy

print(f'Train Accuracy: {evaluate_model(model, train_loader)}')
print(f'Test Accuracy: {evaluate_model(model, test_loader)}')

C:\Users\jlari\AppData\Local\Temp\ipykernel_32320\3048843464.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_training['subject'] = label_encoder.fit_transform(df_training['subject'])


Epoch 1, Loss: 0.03574181720614433
Epoch 2, Loss: 0.06873874366283417
Epoch 3, Loss: 0.07781776040792465
Epoch 4, Loss: 0.12707112729549408
Epoch 5, Loss: 0.3069974184036255
Epoch 6, Loss: 0.27999722957611084
Epoch 7, Loss: 0.03504602611064911
Epoch 8, Loss: 0.07471232861280441
Epoch 9, Loss: 1.077515721321106
Epoch 10, Loss: 0.006855769548565149
Train Accuracy: 0.9455704660615847
Test Accuracy: 0.948663697104677


Would adding a hidden layer improve the accuracy?

In [21]:
class TextClassifier_hiddenlayer(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(TextClassifier_hiddenlayer, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)  # First layer (input to hidden)
        self.relu = nn.ReLU()  # Activation function for the hidden layer
        self.fc2 = nn.Linear(hidden_dim, 1)  # Second layer (hidden to output)
        self.sigmoid = nn.Sigmoid()  # Sigmoid for binary classification
    
    def forward(self, x):
        x = self.fc1(x)  # Pass input through the first linear layer (input to hidden)
        x = self.relu(x)  # Apply ReLU activation
        x = self.fc2(x)  # Pass through the second linear layer (hidden to output)
        x = self.sigmoid(x)  # Apply sigmoid activation for output
        return x
    
input_dim = X_train.shape[1]
hidden_dim = 100  # Example value, you can adjust it
model_2 = TextClassifier_hiddenlayer(input_dim, hidden_dim)

criterion = nn.BCELoss()  # loss function minimise 
optimizer = optim.Adam(model_2.parameters(), lr=0.001)  # optimizer - lr learning rate, a critical hyperparameter that controls how large or small the updates to the weights will be at each step
# Adam optimizer

# Training loop
for epoch in range(5):  # choose epochs - one complete pass through the entire dataset
    model_2.train()
    for features, labels in train_loader:
        optimizer.zero_grad() #  reset the gradients for each new epoch 
        outputs = model_2(features) 
        loss = criterion(outputs.squeeze(), labels.float())  # compute the loss
        loss.backward()
        optimizer.step() # update the weights 
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

print(f'Train Accuracy: {evaluate_model(train_loader)}')
print(f'Test Accuracy: {evaluate_model(test_loader)}')

Epoch 1, Loss: 0.10727664083242416
Epoch 2, Loss: 0.00014839951472822577
Epoch 3, Loss: 0.009782358072698116
Epoch 4, Loss: 0.00034248080919496715
Epoch 5, Loss: 0.00020904683333355933
Train Accuracy: 0.9564563728492678
Test Accuracy: 0.9555679287305122


No, but the loss function does decrease more reliably.